<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Read-in-Data" data-toc-modified-id="Read-in-Data-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Read in Data</a></span><ul class="toc-item"><li><span><a href="#Read-in-excel-files-combining-ticker-symbols-with-the-IQID" data-toc-modified-id="Read-in-excel-files-combining-ticker-symbols-with-the-IQID-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Read in excel files combining ticker symbols with the IQID</a></span></li><li><span><a href="#Read-in-independent-variables,-join-tickers" data-toc-modified-id="Read-in-independent-variables,-join-tickers-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Read in independent variables, join tickers</a></span></li><li><span><a href="#Join-in-the-credit-rating-data" data-toc-modified-id="Join-in-the-credit-rating-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Join in the credit rating data</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd 
import numpy as np 
import plotly.graph_objects as go 
from tqdm import tqdm 

## Read in Data

---
### Read in excel files combining ticker symbols with the IQID 

In [2]:
ids= pd.DataFrame() 
## Read in ticker symbols
for i in range(1, 6): 
    df = pd.read_excel('capiq_data/in_process_ids/ids {}.xlsx'.format(i),
                       engine='openpyxl')[['ID', 'IQID', 'IQ Name']]
    ids = pd.concat([ids, df]) 

## See if there are any duplicates 
print(ids.duplicated().sum()) 
## See if there are any nulls 
print(ids.isna().sum()) 
ids.head() 

0
ID         0
IQID       0
IQ Name    0
dtype: int64


,ID,IQID,IQ Name
0,MMM,IQ289194,3M Company
1,ABT,IQ247483,Abbott Laboratories
2,ABBV,IQ141885706,AbbVie Inc.
3,ABMD,IQ247589,"Abiomed, Inc."
4,ACN,IQ972190,Accenture plc


### Read in independent variables, join tickers 

In [3]:
## Join IQID for inds so we can see the Ticker and the name 

ind_df = pd.read_csv('small_df.csv') 

ind_df = ind_df.merge(ids, on = ['IQID', 'IQ Name']) 
ind_df.drop(['Unnamed: 0', 'IQ Name', 'IQID', 
            'quarter'], axis = 1, inplace = True) 

## The data is quarterly, but we need to lok at it annually. 
## This means we have to take the mean of the data for all the quarters 
ind_df = ind_df.groupby(['year', 'ID']).mean().reset_index()
                      
ind_df.head() 

,year,ID,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
0,2000,PYPL,4.262073,-44.73064,0.0,-11.11638,77.265667,2.933000,34.536333,0.076667,25.746083,42.729333,36.093333
1,2001,PYPL,26.078250,-26.95056,0.0,2.38625,198.586997,5.876978,126.882032,1.314882,3.115630,103.751965,101.747015
2,2002,PYPL,51.266500,0.88000,0.0,16.28150,265.305500,10.945000,107.277000,5.615000,34.556500,158.028500,89.832500
3,2006,HLT,2219.500000,162.00000,705.0,667.00000,16737.000000,1112.000000,13195.500000,7901.000000,-771.500000,3541.500000,126.000000
4,2007,HLT,1974.500000,130.00000,690.0,593.50000,16127.500000,1745.000000,12126.500000,6869.500000,-666.000000,4001.000000,149.000000


### Join in the credit rating data

We want an inner join so we only keep the companies that we have the credit rating for

In [4]:
## Join in the credit Ratings Data
credit_ratings = pd.read_csv('credit.csv')[['Year', 'TickerSymbol',
                                            'DomesticLTICRSPMthlyAvg']]
credit_ratings.rename({'DomesticLTICRSPMthlyAvg': 'rating'}, inplace = True, axis = 1)  

## Add credit ratings to df  
tot_df = credit_ratings.merge(ind_df, how = 'inner', 
                             left_on = ['Year', 'TickerSymbol'], 
                             right_on = ['year', 'ID'])
tot_df.drop(['Year', 'TickerSymbol'], axis = 1, inplace = True) 

tot_df.head() 

,rating,year,ID,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
0,18.000000,2010,AAL,5586.00,-98.00,738.00,1286.00,25088.00,1156.00,29033.00,11136.0,349.00,-3945.00,168.00
1,19.500000,2011,AAL,5994.75,-494.75,923.50,1181.00,25366.75,1301.00,30467.25,11580.5,181.75,-5100.50,297.75
2,27.000000,2012,AAL,6213.75,-469.00,1117.25,1374.25,24168.50,1697.75,32422.50,10375.0,120.75,-8254.00,433.50
3,26.166667,2013,AAL,6678.00,-458.50,1379.75,1729.50,29781.50,1703.25,36592.75,13172.5,949.75,-6811.25,766.75
4,16.833333,2014,AAL,10662.25,720.50,1930.25,3009.50,43986.50,1543.50,40943.25,17203.0,-78.75,3043.25,1160.25


In [5]:
tot_df_clean = pd.DataFrame() 



for ticker in tqdm(tot_df['ID'].unique()): 
    
    
    ## small df is all the rows with the ticker, sort by year 
    small_df = tot_df[tot_df['ID'] == ticker].sort_values(by = 'year',
                                                             ascending = True)
    
    
    ## Insert a lead rating column. This is the predictor column, 
    ## as we are trying to predict the credit rating for the next year
    small_df.insert(loc = 0, column = 'lead_rating', 
               value = small_df.rating.shift(1)) 
    
    
    ## Set the index as the year and the ticker. 
    small_df.set_index(['year', 'ID'], inplace = True) 
    
    ## Take the difference between rows. We are looking to find 
    ## differences in credit rating, so we are going to compare it to 
    ## differences in dependent variables. 
    ## We can then drop the nulls. 
    small_df = small_df.diff().dropna()  
    
    tot_df_clean = pd.concat([tot_df_clean, small_df], axis = 0) 

tot_df_clean.head() 

100%|██████████| 306/306 [00:01<00:00, 201.11it/s]


,,lead_rating,rating,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
year,ID,,,,,,,,,,,,,
2012,AAL,1.500000,7.500000,219.00000,25.75000,193.75000,193.25000,-1198.25000,396.7500,1955.2500,-1205.50000,-61.00000,-3153.50000,135.75000
2013,AAL,7.500000,-0.833333,464.25000,10.50000,262.50000,355.25000,5613.00000,5.5000,4170.2500,2797.50000,829.00000,1442.75000,333.25000
2014,AAL,-0.833333,-9.333333,3984.25000,1179.00000,550.50000,1280.00000,14205.00000,-159.7500,4350.5000,4030.50000,-1028.50000,9854.50000,393.50000
2012,PNW,-0.583333,-0.583333,15.10625,10.51725,-28.39100,23.65400,549.96775,-15.3985,382.2985,-186.43050,28.66800,167.66925,-166.78575
2013,PNW,-0.583333,-0.916667,38.20600,6.13300,30.82575,2.98675,239.18900,-0.1955,-19.5145,-71.94575,36.23175,258.70350,15.06900


In [34]:
from sklearn.model_selection import train_test_split 

## Split into x and y
x = tot_df_clean.drop(['lead_rating'], axis = 1) 

lead_rating = tot_df_clean['lead_rating'].to_numpy()

lead_rating[lead_rating == 0] = 0
lead_rating[(lead_rating > 0) & (lead_rating <= 1)] = 1 
lead_rating[(lead_rating < 0) & (lead_rating >= -1)] = -1 
lead_rating[lead_rating > 1] = 2 
lead_rating[lead_rating < -1] = -2
tot_df_clean['lead_rating'] = lead_rating

y = tot_df_clean['lead_rating'] 


## Normalize the data, but we don't need to normalize the 
## dependent variable
x = (x - x.mean()) / (x.std())

## We'll set aside 5% of the data for testing
train_x, test_x, train_y, test_y = train_test_split(x, y, train_size = 0.95)

In [35]:
from sklearn.model_selection import RepeatedStratifiedKFold 
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV



params = {'kernel':('linear', 'rbf'), 'C':[1, 10]}

## define evaluation
cv = RepeatedStratifiedKFold(n_splits = 10, 
                            n_repeats = 3)


search = GridSearchCV(estimator = SVC(), 
                      param_grid = params, verbose = 1)
search.fit(x, y)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 10], 'kernel': ('linear', 'rbf')}, verbose=1)

In [16]:
from sklearn.metrics import r2_score

prediction = search.predict(test_x)
r2_score(test_y, prediction)  

0.06640130188690418

In [33]:
print(tot_df_clean[tot_df_clean['lead_rating'] == 0].shape) 
print(tot_df_clean[tot_df_clean['lead_rating'].between(0.0001, 1)].shape)
print(tot_df_clean[tot_df_clean['lead_rating'].between(-1, -0.0001)].shape)



(499, 13)
(71, 13)
(148, 13)


array([ 2.,  2., -1., -1., -1., -1.,  0.,  1.,  2., -1., -1.,  2.,  0.,
        0., -2., -1., -1.,  0.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  1., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  1.,  1.,  1.,
        1.,  1.,  0.,  1.,  1.,  2., -1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0.,  0.,  1.,  1.,  0., -2., -1., -1., -1.,  2.,  1., -1.,  0.,
       -1.,  0.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,
        0., -1., -1.,  1.,  0.,  0.,  0.,  0., -1., -1.,  0.,  0.,  0.,
        0.,  0., -1.,  0.,  0.,  0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,
       -1., -1., -1.,  0.,  0.,  0.,  0., -1., -2., -2.,  0.,  0.,  0.,
        0.,  0.,  0.,  0., -1., -1.,  0.,  0., -2., -1.,  0.,  0.,  0.,
        0.,  0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  1.,  0., -1.,
       -1.,  0.,  0.,  0.,  2.,  0.,  0.,  0.,  0., -1.,  0.,  0

In [22]:
tot_df_clean

,,lead_rating,rating,IQ_TOTAL_REV,IQ_NI_CF,IQ_AR,IQ_GP,IQ_TOTAL_ASSETS,IQ_AP,IQ_TOTAL_LIAB,IQ_TOTAL_DEBT,IQ_CASH_FINAN,IQ_TOTAL_EQUITY,IQ_CASH_EQUIV
year,ID,,,,,,,,,,,,,
2012,AAL,1.500000,7.500000,219.00000,25.75000,193.75000,193.25000,-1198.25000,396.7500,1955.2500,-1205.50000,-61.00000,-3153.50000,135.75000
2013,AAL,7.500000,-0.833333,464.25000,10.50000,262.50000,355.25000,5613.00000,5.5000,4170.2500,2797.50000,829.00000,1442.75000,333.25000
2014,AAL,-0.833333,-9.333333,3984.25000,1179.00000,550.50000,1280.00000,14205.00000,-159.7500,4350.5000,4030.50000,-1028.50000,9854.50000,393.50000
2012,PNW,-0.583333,-0.583333,15.10625,10.51725,-28.39100,23.65400,549.96775,-15.3985,382.2985,-186.43050,28.66800,167.66925,-166.78575
2013,PNW,-0.583333,-0.916667,38.20600,6.13300,30.82575,2.98675,239.18900,-0.1955,-19.5145,-71.94575,36.23175,258.70350,15.06900
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2014,PM,0.000000,0.000000,-362.50000,-270.75000,52.75000,-369.00000,-1617.75000,64.7500,2392.5000,2977.75000,344.00000,-4010.25000,-1503.50000
2013,HII,0.000000,0.000000,28.00000,28.75000,214.50000,27.75000,227.75000,-29.5000,123.0000,-35.25000,-35.25000,104.75000,42.50000
2014,HII,0.000000,-1.083333,34.25000,19.25000,54.75000,61.00000,-62.75000,-15.2500,-593.0000,-66.00000,-40.75000,530.25000,-30.00000
